# Who Wants Shots?
<center> <img src="./Images/myriam-zilles-Ep941iDLwhk-unsplash.jpg" alt="Sick Stuffy" width="800"/> </center>
<center> Photo by <a href="https://unsplash.com/@myriamzilles?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Myriam Zilles</a> on <a href="https://unsplash.com/collections/IUPhDJeZav4/flu-vaccination-social?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText">Unsplash</a> </center>
  
### Not that kind! 


Utilizing data courtesy of the United States National Center for Health Statistics, I hope to be able to predict whether or not a given citizen will choose to be vaccinated based on existing patterns from past survey results. 

Business Problem: Guiding public health efforts with regard to vaccination status based on underlying patterns in the opinions, backgrounds, educations and other factors in the general population.

Stakeholder:


## Data Understanding
Here we have a dataset provided by the US National Center for Health Statistics. The merged dataframe consists of 26,707 survey responses to 37 questions. Some questions were not answered by the respondents. 

Columns represent the questions asked in the survey, while rows represent the survey respondents. 

Survey questions/column headers are as follows:
 * h1n1_concern - Level of concern about the H1N1 flu. 
     * Ranked from 0 (not at all concerned) to 3 (very concerned).
 * h1n1_knowledge - Level of knowledge about H1N1 flu. 
     * Ranked from 0 (no knowledge) to 2 (a lot of knowledge).
 * behavioral_antiviral_meds - Has respondent taken antiviral medication? Yes/No (binary)
 * behavioral_avoidance - Has respondent avoided close contact with others with flu-like symptoms? Yes/No (binary)
 * behavioral_face_mask - Has respondent bought a face mask? Yes/No (binary)
 * behavioral_wash_hands - Has respondent frequently washed hands or used hand sanitizer? Yes/No (binary)
 * behavioral_large_gatherings - Has respondent reduced time at large gatherings? Yes/No (binary)
 * behavioral_outside_home - Has respondent reduced contact with people outside of own household? Yes/No (binary)
 * behavioral_touch_face - Has respondent avoided touching face(including eyes, nose, mouth, etc)? Yes/No (binary)
 * doctor_recc_h1n1 - Was H1N1 flu vaccine recommended by doctor? Yes/No (binary)
 * doctor_recc_seasonal - Was Seasonal flu vaccine recommended by doctor? Yes/No (binary)
 * chronic_med_condition - Does respondent have chronic medical condition? Yes/No (binary) Conditions include: asthma or other lung condition, diabetes, heart condition, kidney condition, sickle cell anemia or other anemia, a neurological or neuromuscular condition, liver condition, or a weakened immune system caused by a chronic illness or by medicines taken for a chronic illness. 
 * child_under_6_months - Does respondent have regular close contact with a child under the age of six months? Yes/No (binary)
 * health_worker - Is respondent a healthcare worker? Yes/No (binary)
 * health_insurance - Does respondent have health insurance? Yes/No (binary)
 * opinion_h1n1_vacc_effective - Respondent's personal opinion about H1N1 vaccine effectiveness
     * Ranked from 1 (not at all effective) to 5 (very effective).
 * opinion_h1n1_risk - Respondent's personal opinion about risk of getting sick with H1N1 flu without vaccine 
     * Ranked from 1 (very low) to 5 (very high)
 * opinion_h1n1_sick_from_vacc - Respondent's worry of getting sick from taking H1N1 vaccine 
     * Ranked from 1 (not at all worried) to 5 (very worried)
 * opinion_seas_vacc_effective - Respondent's personal opinion about seasonal flu vaccine effectiveness 
     * Ranked from 1 (not at all effective) to 5 (very effective)
 * opinion_seas_risk - Respondent's personal opinion about risk of getting sick with seasonal flu without vaccine 
     * Ranked from 1 (very low) to 5 (very high)
 * opinion_seas_sick_from_vacc - Respondent's worry of getting sick from taking seasonal flu vaccine 
     * Ranked from 1 (not at all worried) to 5 (very worried).
 * age_group - Age group of respondent - Ranges are 18-34, 35-44, 45-54, 55-64, and 65+ Years.
 * education - Self-reported education level. Less than 12 years, 12 years, Some College, or College Graduate.
 * race - Race of respondent. White, Black, Hispanic, and Other/Multiple.
 * sex - Sex of respondent. Male or Female 
 * income_poverty - Respondent annual household income with respect to 2008 Census poverty thresholds.
     * "Below Poverty", "<= \\$75,000 Above Poverty" , or "> \\$75,000"
 * marital_status - Marital status of respondent.
     * Married or Not Married
 * rent_or_own - Housing situation of respondent.
     * Rent or Own
 * employment_status - Employment status of respondent.
     * Employed, Not in Labor Force, or Unemployed
 * hhs_geo_region - Respondent's residence using a 10-region geographic classification defined by the U.S. Dept. of Health and Human Services. Values are represented as short random character strings.
 * census_msa - Respondent's residence within metropolitan statistical areas (MSA) as defined by the U.S. Census.
 * household_adults - Number of OTHER adults in household, responses range 0 through 3.
 * household_children - Number of children in household, responses range 0 to 3.
 * employment_industry - Industry respondent is employed in. Values are represented as random, encoded character strings.
 * employment_occupation - Occupation of respondent. Values are represented as random, encoded character strings.

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt 
%matplotlib inline


In [2]:
labels = pd.read_csv('Data/training_set_labels.csv')
labels

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0
...,...,...,...
26702,26702,0,0
26703,26703,0,0
26704,26704,0,1
26705,26705,0,0


In [3]:
features = pd.read_csv('Data/training_set_features.csv')
features

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Married,Rent,Employed,lrircsnp,Non-MSA,1.0,0.0,fcxhlnwr,haliazsg


In [4]:
#merge labels and features together in one DF
df = pd.merge(features,labels, on='respondent_id')
df.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0


In [5]:
#figure out what's goin on in here... looks like I've got some objects and nulls to deal with.
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                26707 non-null  int64  
 1   h1n1_concern                 26615 non-null  float64
 2   h1n1_knowledge               26591 non-null  float64
 3   behavioral_antiviral_meds    26636 non-null  float64
 4   behavioral_avoidance         26499 non-null  float64
 5   behavioral_face_mask         26688 non-null  float64
 6   behavioral_wash_hands        26665 non-null  float64
 7   behavioral_large_gatherings  26620 non-null  float64
 8   behavioral_outside_home      26625 non-null  float64
 9   behavioral_touch_face        26579 non-null  float64
 10  doctor_recc_h1n1             24547 non-null  float64
 11  doctor_recc_seasonal         24547 non-null  float64
 12  chronic_med_condition        25736 non-null  float64
 13  child_under_6_mo

In [6]:
#employment industry and occupation appear to be encoded for anonymity. Will operate under the assumption the
#stakeholder knows the keys to these encoded values
#Nulls should be filled with 'Unknown' so that we can OHE these features for modeling. Employment could be a 
#valuable feature and there are many missing values here.
df.employment_industry.value_counts()

fcxhlnwr    2468
wxleyezf    1804
ldnlellj    1231
pxcmvdjn    1037
atmlpfrs     926
arjwrbjb     871
xicduogh     851
mfikgejo     614
vjjrobsf     527
rucpziij     523
xqicxuve     511
saaquncn     338
cfqqtusy     325
nduyfdeo     286
mcubkhph     275
wlfvacwt     215
dotnnunm     201
haxffmxo     148
msuufmds     124
phxvnwax      89
qnlwzans      13
Name: employment_industry, dtype: int64

In [7]:
#like above, could be important feature - will operate under the assumption of encoded for anonymity and 
#stakeholder knows keys for these values
df.hhs_geo_region.value_counts()

lzgpxyit    4297
fpwskwrf    3265
qufhixun    3102
oxchjgsf    2859
kbazzjca    2858
bhuqouqj    2846
mlyzmhmf    2243
lrircsnp    2078
atmpeygn    2033
dqpwygqj    1126
Name: hhs_geo_region, dtype: int64

In [8]:
#good feature - looks good to OHE
df.income_poverty.value_counts()

<= $75,000, Above Poverty    12777
> $75,000                     6810
Below Poverty                 2697
Name: income_poverty, dtype: int64

In [9]:
df.sex.value_counts()

Female    15858
Male      10849
Name: sex, dtype: int64

In [10]:
df.race.value_counts()

White                21222
Black                 2118
Hispanic              1755
Other or Multiple     1612
Name: race, dtype: int64

In [11]:
df.age_group.value_counts()

65+ Years        6843
55 - 64 Years    5563
45 - 54 Years    5238
18 - 34 Years    5215
35 - 44 Years    3848
Name: age_group, dtype: int64

In [12]:
df.census_msa.value_counts()

MSA, Not Principle  City    11645
MSA, Principle City          7864
Non-MSA                      7198
Name: census_msa, dtype: int64

In [13]:
df.education.value_counts()

College Graduate    10097
Some College         7043
12 Years             5797
< 12 Years           2363
Name: education, dtype: int64

In [14]:
df.isna().sum()

respondent_id                      0
h1n1_concern                      92
h1n1_knowledge                   116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_h1n1_vacc_effective      391
opinion_h1n1_risk                388
opinion_h1n1_sick_from_vacc      395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
m

In [15]:
df.household_adults.value_counts()

1.0    14474
0.0     8056
2.0     2803
3.0     1125
Name: household_adults, dtype: int64

# Data Analysis
Through EDA, there are several object columns that will need to be One Hot Encoded in order to be modeled, including age_group, education, race, sec, marital_status, and employment_industry and occupation, among others. These could turn out to be important features, so I will encode and handle any null values so as to have as much data for modeling as possible. 

There are quite a few missing values, with highest count seen in employment industry and occupation and health insurance. I will add an "Unknown" or "Not Answered" value to encode those missing values so as to not lose any rows. Missing values in the numeric columns will be assessed and filled with mean, median or mode depending on the column and the assessment (TBD). 

### H1N1 Target Modeling

In [16]:
#drop seasonal flu vaccine columns from h1n1 dataframe.
h1_df = df.drop(columns=['opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc',
                 'seasonal_vaccine', 'doctor_recc_seasonal'], axis= 1)
h1_df.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0


In [29]:
h1_df.isna().sum()

respondent_id                      0
h1n1_concern                      92
h1n1_knowledge                   116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_h1n1_vacc_effective      391
opinion_h1n1_risk                388
opinion_h1n1_sick_from_vacc      395
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
marital_status                  1408
rent_or_own                     2042
employment_status               1463
hhs_geo_region                     0
c

In [103]:
y = h1_df['h1n1_vaccine']
X = h1_df.drop('h1n1_vaccine', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=14)

In [104]:
X_train.describe()
#fill na strategy - fill na with median value (0 or 1) for binary missing values and median for numeric columns. 
#fill na strategy for object columns: fill with "unknown" so OHE adds a column for those 

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,household_adults,household_children
count,20030.000000,19960.000000,19941.000000,19980.000000,19872.000000,20015.000000,20005.000000,19969.000000,19971.000000,19936.000000,18438.000000,19293.000000,19412.000000,19420.000000,10919.000000,19730.000000,19733.000000,19722.000000,19839.000000,19839.000000
mean,13350.502396,1.616683,1.263427,0.048348,0.724487,0.069898,0.823994,0.357104,0.336388,0.675261,0.222041,0.286788,0.082835,0.112358,0.879201,3.848555,2.342067,2.355390,0.886385,0.537124
std,7697.832411,0.911216,0.617174,0.214507,0.446784,0.254981,0.380835,0.479158,0.472485,0.468289,0.415630,0.452273,0.275640,0.315815,0.325908,1.008089,1.286795,1.362349,0.752651,0.930719
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,6701.250000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,1.000000,1.000000,0.000000,0.000000
50%,13330.500000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000,2.000000,2.000000,1.000000,0.000000
75%,19997.750000,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,5.000000,4.000000,4.000000,1.000000,1.000000
max,26706.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,5.000000,5.000000,3.000000,3.000000


In [105]:
X_train.child_under_6_months.mode()

0    0.0
dtype: float64

In [114]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20030 entries, 20852 to 19051
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                20030 non-null  int64  
 1   h1n1_concern                 19960 non-null  float64
 2   h1n1_knowledge               19941 non-null  float64
 3   behavioral_antiviral_meds    19980 non-null  float64
 4   behavioral_avoidance         19872 non-null  float64
 5   behavioral_face_mask         20015 non-null  float64
 6   behavioral_wash_hands        20005 non-null  float64
 7   behavioral_large_gatherings  19969 non-null  float64
 8   behavioral_outside_home      19971 non-null  float64
 9   behavioral_touch_face        19936 non-null  float64
 10  doctor_recc_h1n1             18438 non-null  float64
 11  chronic_med_condition        19293 non-null  float64
 12  child_under_6_months         19412 non-null  float64
 13  health_worke

In [107]:
obj_cols = ['age_group', 'education', 'race', 'sex', 'income_poverty', 'marital_status', 
        'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa', 'employment_industry', 
        'employment_occupation']
X_train[obj_cols].fillna(value='Unknown')

,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,employment_industry,employment_occupation
20852,45 - 54 Years,Some College,White,Female,"> $75,000",Married,Own,Employed,bhuqouqj,"MSA, Not Principle City",fcxhlnwr,cmhcxjea
8888,18 - 34 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,lrircsnp,"MSA, Not Principle City",fcxhlnwr,cmhcxjea
20776,18 - 34 Years,12 Years,White,Female,Unknown,Not Married,Unknown,Employed,mlyzmhmf,"MSA, Not Principle City",pxcmvdjn,mxkfnird
11901,55 - 64 Years,Some College,White,Female,"> $75,000",Married,Own,Employed,qufhixun,Non-MSA,fcxhlnwr,cmhcxjea
24470,18 - 34 Years,Unknown,White,Male,Unknown,Unknown,Unknown,Unknown,lzgpxyit,Non-MSA,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
22855,65+ Years,Some College,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,kbazzjca,"MSA, Principle City",Unknown,Unknown
18838,65+ Years,Some College,White,Female,Unknown,Married,Own,Not in Labor Force,fpwskwrf,"MSA, Not Principle City",Unknown,Unknown
9484,55 - 64 Years,12 Years,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,oxchjgsf,"MSA, Not Principle City",xqicxuve,rcertsgn
13656,65+ Years,12 Years,White,Female,"> $75,000",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",Unknown,Unknown


In [108]:
X_train.isna().sum()

respondent_id                      0
h1n1_concern                      70
h1n1_knowledge                    89
behavioral_antiviral_meds         50
behavioral_avoidance             158
behavioral_face_mask              15
behavioral_wash_hands             25
behavioral_large_gatherings       61
behavioral_outside_home           59
behavioral_touch_face             94
doctor_recc_h1n1                1592
chronic_med_condition            737
child_under_6_months             618
health_worker                    610
health_insurance                9111
opinion_h1n1_vacc_effective      300
opinion_h1n1_risk                297
opinion_h1n1_sick_from_vacc      308
age_group                          0
education                       1054
race                               0
sex                                0
income_poverty                  3310
marital_status                  1064
rent_or_own                     1540
employment_status               1100
hhs_geo_region                     0
c

In [110]:
cat_cols = ['age_group', 'education', 'race', 'sex',
       'income_poverty', 'marital_status', 'rent_or_own', 'employment_status',
       'hhs_geo_region', 'census_msa','employment_industry', 'employment_occupation']
cat_imp = SimpleImputer(strategy="constant", fill_value="Unknown")
cat_imp.fit(X_train[cat_cols])
X_train_cats = pd.DataFrame(cat_imp.transform(X_train[cat_cols]), columns = X_train[cat_cols].columns)
X_train_cats

,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,employment_industry,employment_occupation
0,45 - 54 Years,Some College,White,Female,"> $75,000",Married,Own,Employed,bhuqouqj,"MSA, Not Principle City",fcxhlnwr,cmhcxjea
1,18 - 34 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,lrircsnp,"MSA, Not Principle City",fcxhlnwr,cmhcxjea
2,18 - 34 Years,12 Years,White,Female,Unknown,Not Married,Unknown,Employed,mlyzmhmf,"MSA, Not Principle City",pxcmvdjn,mxkfnird
3,55 - 64 Years,Some College,White,Female,"> $75,000",Married,Own,Employed,qufhixun,Non-MSA,fcxhlnwr,cmhcxjea
4,18 - 34 Years,Unknown,White,Male,Unknown,Unknown,Unknown,Unknown,lzgpxyit,Non-MSA,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
20025,65+ Years,Some College,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,kbazzjca,"MSA, Principle City",Unknown,Unknown
20026,65+ Years,Some College,White,Female,Unknown,Married,Own,Not in Labor Force,fpwskwrf,"MSA, Not Principle City",Unknown,Unknown
20027,55 - 64 Years,12 Years,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,oxchjgsf,"MSA, Not Principle City",xqicxuve,rcertsgn
20028,65+ Years,12 Years,White,Female,"> $75,000",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",Unknown,Unknown


In [111]:
X_train_cats.isna().sum()

age_group                0
education                0
race                     0
sex                      0
income_poverty           0
marital_status           0
rent_or_own              0
employment_status        0
hhs_geo_region           0
census_msa               0
employment_industry      0
employment_occupation    0
dtype: int64

In [115]:
num_cols = ['respondent_id','h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'chronic_med_condition',
       'child_under_6_months', 'health_worker', 'health_insurance',
       'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc','household_adults',
       'household_children']
num_imp = SimpleImputer(strategy='median')
num_imp.fit(X_train[num_cols])
X_train_nums = pd.DataFrame(num_imp.transform(X_train[num_cols]), columns = X_train[num_cols].columns)
X_train_nums

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,household_adults,household_children
0,20852.0,3.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,5.0,1.0,1.0,0.0,0.0
1,8888.0,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,4.0,4.0,4.0,1.0,3.0
2,20776.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,2.0,4.0,3.0,0.0
3,11901.0,3.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,4.0,1.0,1.0,0.0
4,24470.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20025,22855.0,1.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,5.0,1.0,1.0,0.0,0.0
20026,18838.0,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,4.0,4.0,0.0,0.0
20027,9484.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,0.0,0.0
20028,13656.0,2.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,5.0,4.0,4.0,1.0,0.0


In [117]:
X_train = pd.concat([X_train_nums, X_train_cats], axis=1)
X_train

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,employment_industry,employment_occupation
0,20852.0,3.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,White,Female,"> $75,000",Married,Own,Employed,bhuqouqj,"MSA, Not Principle City",fcxhlnwr,cmhcxjea
1,8888.0,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,lrircsnp,"MSA, Not Principle City",fcxhlnwr,cmhcxjea
2,20776.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,White,Female,Unknown,Not Married,Unknown,Employed,mlyzmhmf,"MSA, Not Principle City",pxcmvdjn,mxkfnird
3,11901.0,3.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,White,Female,"> $75,000",Married,Own,Employed,qufhixun,Non-MSA,fcxhlnwr,cmhcxjea
4,24470.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,White,Male,Unknown,Unknown,Unknown,Unknown,lzgpxyit,Non-MSA,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20025,22855.0,1.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,kbazzjca,"MSA, Principle City",Unknown,Unknown
20026,18838.0,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,White,Female,Unknown,Married,Own,Not in Labor Force,fpwskwrf,"MSA, Not Principle City",Unknown,Unknown
20027,9484.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,oxchjgsf,"MSA, Not Principle City",xqicxuve,rcertsgn
20028,13656.0,2.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,White,Female,"> $75,000",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",Unknown,Unknown


In [118]:
X_train.isna().sum()

respondent_id                  0
h1n1_concern                   0
h1n1_knowledge                 0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_h1n1               0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_h1n1_vacc_effective    0
opinion_h1n1_risk              0
opinion_h1n1_sick_from_vacc    0
household_adults               0
household_children             0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_region                 0
census_msa                     0
employment

In [123]:
X_test_cats = pd.DataFrame(cat_imp.transform(X_test[cat_cols]), columns = X_test[cat_cols].columns)

In [124]:
X_test_nums = pd.DataFrame(num_imp.transform(X_test[num_cols]), columns = X_test[num_cols].columns)

In [125]:
X_test = pd.concat([X_test_nums, X_test_cats], axis=1)
X_test

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,employment_industry,employment_occupation
0,9706.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,White,Female,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,dqpwygqj,"MSA, Not Principle City",Unknown,Unknown
1,16888.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,White,Male,"> $75,000",Married,Rent,Employed,bhuqouqj,Non-MSA,xicduogh,xtkaffoo
2,15593.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,Hispanic,Male,Unknown,Not Married,Rent,Employed,kbazzjca,"MSA, Not Principle City",xicduogh,qxajmpny
3,17295.0,2.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,Black,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,kbazzjca,"MSA, Not Principle City",wxleyezf,bxpfxfdn
4,21698.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,Black,Male,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",vjjrobsf,rcertsgn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6672,19123.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,atmpeygn,"MSA, Not Principle City",wxleyezf,dlvbwzss
6673,312.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,White,Female,"> $75,000",Married,Own,Not in Labor Force,bhuqouqj,"MSA, Not Principle City",Unknown,Unknown
6674,17992.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,lzgpxyit,"MSA, Not Principle City",atmlpfrs,vlluhbov
6675,23950.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Unemployed,lzgpxyit,"MSA, Not Principle City",Unknown,Unknown


In [161]:
X_train_dum = pd.get_dummies(X_train, columns=cat_cols)
# X_train = pd.concat([X_train, X_train_dum], axis=1)
# X_train.head()

### Seasonal Flu Target Modeling

In [18]:
#drop h1n1 columns from seasonal flu df
flu_df = df.drop(columns=['h1n1_concern','h1n1_knowledge','doctor_recc_h1n1', 
                'opinion_h1n1_vacc_effective','opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc', 'h1n1_vaccine'],
                axis=1)
flu_df.head()

,respondent_id,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_seasonal,chronic_med_condition,...,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,seasonal_vaccine
0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0
1,1,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,1
2,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,...,Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0
3,3,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,1
4,4,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0


In [19]:
flu_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                26707 non-null  int64  
 1   behavioral_antiviral_meds    26636 non-null  float64
 2   behavioral_avoidance         26499 non-null  float64
 3   behavioral_face_mask         26688 non-null  float64
 4   behavioral_wash_hands        26665 non-null  float64
 5   behavioral_large_gatherings  26620 non-null  float64
 6   behavioral_outside_home      26625 non-null  float64
 7   behavioral_touch_face        26579 non-null  float64
 8   doctor_recc_seasonal         24547 non-null  float64
 9   chronic_med_condition        25736 non-null  float64
 10  child_under_6_months         25887 non-null  float64
 11  health_worker                25903 non-null  float64
 12  health_insurance             14433 non-null  float64
 13  opinion_seas_vac